In [1]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 261 kB in 3s (78.0 kB/s)
Reading package li

In [2]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-12-30 05:07:56--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.2’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.68MB/s    in 0.2s    

2022-12-30 05:07:57 (5.68 MB/s) - ‘postgresql-42.2.9.jar.2’ saved [914037/914037]



In [3]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [45]:
from config import password

In [4]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles

url = "https://amazonsampledata.s3.amazonaws.com/amazon_reviews_us_Electronics_v1_00.csv"

spark.sparkContext.addFile(url)
df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Electronics_v1_00.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")

# Show DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+----+----+----+----+----+----+----+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|_c15|_c16|_c17|_c18|_c19|_c20|_c21|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+----+----+----+----+----+----+----+
|         US|   41409413|R2MTG1GCZLR2DK|B00428R89M|     112201306|yoomall 5M Antenn...|     Electronics|          5|            0|          0|   N|                Y|          Five Stars|       As described.| 2015-08-31|null|null|null|n

In [17]:
# Get the number of rows in the DataFrame.
df.count()

1002294

In [18]:
len(df.columns)

22

# Transform the Data

## Create the "review_id_table".

In [21]:
from pyspark.sql.functions import to_date
# Create the "review_id_df" DataFrame with the appropriate columns and data types.

review_id_df = df.select(['review_id','customer_id','product_id','product_parent','review_date'])
review_id_df.show(5)



+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R2MTG1GCZLR2DK|   41409413|B00428R89M|     112201306| 2015-08-31|
|R2HBOEM8LE9928|   49668221|B000068O48|     734576678| 2015-08-31|
|R1P4RW1R9FDPEE|   12338275|B000GGKOG8|     614448099| 2015-08-31|
|R1EBPM82ENI67M|   38487968|B000NU4OTA|      72265257| 2015-08-31|
|R372S58V6D11AT|   23732619|B00JOQIO6S|     308169188| 2015-08-31|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



## Create the "products" Table

In [24]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 

products_df = df.select(['product_id','product_title'])
products_df.show(5)


+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00428R89M|yoomall 5M Antenn...|
|B000068O48|"Hosa GPM-103 3.5...|
|B000GGKOG8|Channel Master Ti...|
|B000NU4OTA|LIMTECH Wall char...|
|B00JOQIO6S|Skullcandy Air Ra...|
+----------+--------------------+
only showing top 5 rows



In [34]:
print(products_df.count())

1002294


In [35]:
products_df = products_df.dropna()

In [36]:
print(products_df.count())

1000326


In [37]:
products_df = products_df.dropDuplicates()

In [38]:
print(products_df.count())

94694


## Create the "customers" Table

In [26]:
customers_df = df.groupby('customer_id').agg({'customer_id': 'count'})
customers_df.show(5)


+-----------+------------------+
|customer_id|count(customer_id)|
+-----------+------------------+
|   11608395|                 1|
|   22848554|                 1|
|   43490052|                 2|
|   16281356|                 1|
|    3223564|                 1|
+-----------+------------------+
only showing top 5 rows



In [30]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 
from pyspark.sql.functions import desc
customers_df = customers_df.withColumnRenamed("count(customer_id)", "customer_count")
customers_df.orderBy(desc('customer_count')).show(5)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   32038204|           136|
|   14308733|            89|
|   51812418|            87|
|   14945970|            85|
|   52870270|            80|
+-----------+--------------+
only showing top 5 rows



In [32]:
customers_df.show(5)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   11608395|             1|
|   22848554|             1|
|   43490052|             2|
|   16281356|             1|
|    3223564|             1|
+-----------+--------------+
only showing top 5 rows



## Create the "vine_table".

In [33]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 

vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes","vine"])
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R2MTG1GCZLR2DK|          5|            0|          0|   N|
|R2HBOEM8LE9928|          5|            0|          0|   N|
|R1P4RW1R9FDPEE|          5|            1|          1|   N|
|R1EBPM82ENI67M|          1|            0|          0|   N|
|R372S58V6D11AT|          5|            1|          1|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



# Load

In [46]:
mode = "overwrite"
jdbc_url="jdbc:postgresql://big-data-base.coodiqlcstgq.us-east-1.rds.amazonaws.com:5432/challenge_22_db"
config = {"user":"postgres", "password": password, "driver":"org.postgresql.Driver"}

In [40]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_df', mode=mode, properties=config)

In [42]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='products_df', mode=mode, properties=config)

In [43]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers_df', mode=mode, properties=config)

In [44]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine_df', mode=mode, properties=config)